In [43]:
#importing neccesary libraries
import numpy as np
import pandas as pd

In [ ]:
#creating a function that swaps different rows of matrix
def swap_rows(M, row_index_1, row_index_2):
    #copy the matrix
    M = M.copy()
    # swap indexes (you use a list of indexes to index rows in Numpy)
    M[[row_index_1, row_index_2]] = M[[row_index_2, row_index_1]]
    #return the swapped matrices
    return M




In [9]:
M = np.array([
[1, 3, 6],
[0, -5, 2],
[-4, 5, 8]
])
print(M)

[[ 1  3  6]
 [ 0 -5  2]
 [-4  5  8]]


In [13]:
swap_rows(M,0,2)

array([[-4,  5,  8],
       [ 0, -5,  2],
       [ 1,  3,  6]])

In [ ]:

def get_index_first_non_zero_from_column(M, column, starting_row):
    """
    Find the index of the first non-zero value in a column from a given starting row.

    Parameters:
    - M (numpy.array): The matrix to search.
    - column (int): The column to search.
    - starting_row (int): The row index to start searching from.

    Returns:
    - int: Index of the first non-zero value in the column, or -1 if none found.
    """
    column_array = M[starting_row:, column]
    
    for i, val in enumerate(column_array):
        if not np.isclose(val, 0, atol=1e-5):
            return i + starting_row
    return -1



In [16]:
N = np.array([
[0, 5, -3 ,6 ,8],
[0, 6, 3, 8, 1],
[0, 0, 0, 0, 0],
[0, 0, 0 ,0 ,7],
[0, 2, 1, 0, 4]
]
)
print(N)

[[ 0  5 -3  6  8]
 [ 0  6  3  8  1]
 [ 0  0  0  0  0]
 [ 0  0  0  0  7]
 [ 0  2  1  0  4]]


In [ ]:
get_index_first_non_zero_from_column(N, column = 0, starting_row = 0)

-1

In [45]:
get_index_first_non_zero_from_column(N, column = -1, starting_row = 2)

3

In [46]:
def get_index_first_non_zero_from_row(M, row, augmented=False):
    """
    Returns the index of the first non-zero value in a specific row.
    
    Parameters:
    - M (numpy.array): The matrix to search.
    - row (int): The row to inspect.
    - augmented (bool): If True, ignore the last column (assumed constants).
    
    Returns:
    - int: Index of first non-zero value, or -1 if all are zero.
    """
    M = M.copy()

    if augmented:
        M = M[:, :-1]  # Remove last column if matrix is augmented

    row_array = M[row]  # Get the specific row

    for i, val in enumerate(row_array):
        if not np.isclose(val, 0, atol=1e-5):
            return i

    return -1



In [22]:
print(N)

[[ 0  5 -3  6  8]
 [ 0  6  3  8  1]
 [ 0  0  0  0  0]
 [ 0  0  0  0  7]
 [ 0  2  1  0  4]]


In [55]:
print(f'Output for row 2: {get_index_first_non_zero_from_row(N, 2)}')
print(f'Output for row 3: {get_index_first_non_zero_from_row(N, 3)}')

Output for row 2: -1
Output for row 3: 4


In [56]:
def augmented_matrix(A, B):
    """
    Create an augmented matrix by horizontally stacking two matrices A and B.

    Parameters:
    - A (numpy.array): First matrix.
    - B (numpy.array): Second matrix.

    Returns:
    - numpy.array: Augmented matrix obtained by horizontally stacking A and B.
    """
    augmented_M = np.hstack((A,B))
    return augmented_M

In [57]:
A = np.array([[1,2,3], [3,4,5], [4,5,6]])
B = np.array([[1], [5], [7]])

print(augmented_matrix(A,B))

[[1 2 3 1]
 [3 4 5 5]
 [4 5 6 7]]


In [68]:
def row_echelon_form(A, B):
    # Check for singular matrix
    if np.isclose(np.linalg.det(A), 0):
        return 'singular system'

    # Copy and convert to float
    A = A.astype('float64').copy()
    B = B.astype('float64').copy()

    # Augment the matrix
    M = augmented_matrix(A, B)

    num_rows = len(A)

    for row in range(num_rows):
        pivot_candidate = M[row, row]

        # If pivot is zero, look below for a row to swap
        if np.isclose(pivot_candidate, 0):
            swap_row_index = get_index_first_non_zero_from_column(M, row, row + 1)
            if swap_row_index == -1:
                continue
            M = swap_rows(M, row, swap_row_index)
            pivot = M[row, row]
        else:
            pivot = pivot_candidate

        # Normalize the pivot row
        M[row] = M[row] / pivot

        # Eliminate values below the pivot
        for j in range(row + 1, num_rows):
            value_below_pivot = M[j, row]
            M[j] = M[j] - value_below_pivot * M[row]

    return M


        

            

In [69]:
A = np.array([[1,2,3],[0,1,0], [0,0,5]])
B = np.array([[1], [2], [4]])
row_echelon_form(A,B)

array([[1. , 2. , 3. , 1. ],
       [0. , 1. , 0. , 2. ],
       [0. , 0. , 1. , 0.8]])

In [70]:
def back_substitution(M):
    M = M.copy()
    num_rows = M.shape[0]

    for row in reversed(range(num_rows)):
        substitution_row = M[row]

        # Find pivot index in the row (ignore the last column)
        index = np.argmax(~np.isclose(substitution_row[:-1], 0))

        # Eliminate above
        for j in range(row):
            row_to_reduce = M[j]
            value = row_to_reduce[index]

            # Row operation: row_j = row_j - value * row_i
            row_to_reduce = row_to_reduce - value * substitution_row

            M[j, :] = row_to_reduce

    solution = M[:, -1]
    return solution


In [71]:
def gaussian_elimination(A, B):
    """
    Solve a linear system represented by an augmented matrix using the Gaussian elimination method.

    Parameters:
    - A (numpy.array): Square matrix of size n x n representing the coefficients of the linear system.
    - B (numpy.array): Column matrix of size n x 1 representing the constant terms.

    Returns:
    numpy.array: The solution vector.
    """

    ### START CODE HERE ###

    # Get the matrix in row echelon form
    row_echelon_M = row_echelon_form(A, B)

    # If the system is singular, return the error message
    if isinstance(row_echelon_M, str):
        return row_echelon_M

    # Perform back substitution to get the result
    solution = back_substitution(row_echelon_M)

    ### END CODE HERE ###

    return solution


In [72]:
A = np.array([
    [2, 1, -1],
    [-3, -1, 2],
    [-2, 1, 2]
])
B = np.array([
    [8],
    [-11],
    [-3]
])
solution = gaussian_elimination(A, B)
print(solution)

[ 2.  3. -1.]
